In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import pearsonr

# Function to read temperature data and dates from an Excel file
def read_excel_file(file_path, sheet_name, date_column, temp_column, date_format=None):
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    if date_format:
        dates = pd.to_datetime(df[date_column], format=date_format)
    else:
        dates = pd.to_datetime(df[date_column])
    dates = dates.dt.date  # Convert to just date, ignoring the time
    temperatures = df[temp_column].values
    return dates, temperatures

# Function to perform empirical quantile mapping
def empirical_quantile_mapping(obs, model, n_quantiles=1000):
    quantiles = np.linspace(0, 1, n_quantiles)
    obs_quantiles = np.quantile(obs, quantiles)
    model_quantiles = np.quantile(model, quantiles)
    model_cdf = np.searchsorted(model_quantiles, model, side='right') / n_quantiles
    corrected_model = np.interp(model_cdf, quantiles, obs_quantiles)
    return corrected_model

# Function to perform detrended quantile mapping
def detrended_quantile_mapping(obs, model, n_quantiles=1000):
    obs_trend = np.polyval(np.polyfit(np.arange(len(obs)), obs, 1), np.arange(len(obs)))
    model_trend = np.polyval(np.polyfit(np.arange(len(model)), model, 1), np.arange(len(model)))
    obs_detrended = obs - obs_trend
    model_detrended = model - model_trend
    corrected_model_detrended = empirical_quantile_mapping(obs_detrended, model_detrended, n_quantiles)
    corrected_model = corrected_model_detrended + model_trend
    return corrected_model

# Function to perform distribution mapping
def distribution_mapping(obs, model, dist=norm):
    obs_params = dist.fit(obs)
    model_transformed = dist.cdf(model, *dist.fit(model))
    corrected_model = dist.ppf(model_transformed, *obs_params)
    return corrected_model

# Function to perform local intensity scaling
def local_intensity_scaling(obs, model):
    obs_mean = np.mean(obs)
    model_mean = np.mean(model)
    scaling_factor = obs_mean / model_mean
    corrected_model = model * scaling_factor
    return corrected_model

# Function to calculate statistical measures
def calculate_statistics(obs, model):
    mae = mean_absolute_error(obs, model)
    mse = mean_squared_error(obs, model)
    rmse = np.sqrt(mse)
    corr, _ = pearsonr(obs, model)
    return mae, mse, rmse, corr

# Function to read observed and model data from Excel files
def read_excel_file(file_path, sheet_name, date_column, temp_column, date_format=None):
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    if date_format:
        dates = pd.to_datetime(df[date_column], format=date_format)
    else:
        dates = pd.to_datetime(df[date_column])
    dates = dates.dt.date  # Convert to just date, ignoring the time
    temperatures = df[temp_column].values
    return dates, temperatures

# Define file paths and column names
observed_file_path = 'C:\\Users\\hp\\Desktop\\Location wise data\\data.xlsx'
model_file_path = 'C:\\Users\\hp\\Desktop\\Location wise data\\data.xlsx'
sheet_name = 'Sheet1'
date_column_name = 'Date'
observed_temp_column_name = 'Raw_tmin C'
model_temp_column_name_prebias = 'Bias_tmin C'
model_temp_column_name_1st_bias = 'Obs_tmin'
date_format = '%d-%m-%Y %H:%M'

# Read data from Excel files
observed_dates, observed_data = read_excel_file(observed_file_path, sheet_name, date_column_name, observed_temp_column_name, date_format)
model_dates_prebias, model_data_prebias = read_excel_file(model_file_path, sheet_name, date_column_name, model_temp_column_name_prebias, date_format)
model_dates_1st_bias, model_data_1st_bias = read_excel_file(model_file_path, sheet_name, date_column_name, model_temp_column_name_1st_bias, date_format)

# Check if dates match
if not np.array_equal(observed_dates, model_dates_prebias) or not np.array_equal(observed_dates, model_dates_1st_bias):
    raise ValueError("The dates in the observed and model data do not match.")

# Handle NaN values
observed_data = np.nan_to_num(observed_data, nan=np.nanmean(observed_data))
model_data_prebias = np.nan_to_num(model_data_prebias, nan=np.nanmean(model_data_prebias))
model_data_1st_bias = np.nan_to_num(model_data_1st_bias, nan=np.nanmean(model_data_1st_bias))

# Apply empirical quantile mapping
corrected_model_data_prebias_eqm = empirical_quantile_mapping(observed_data, model_data_prebias, n_quantiles=1000)
corrected_model_data_1st_bias_eqm = empirical_quantile_mapping(observed_data, model_data_1st_bias, n_quantiles=1000)

# Apply detrended quantile mapping
corrected_model_data_prebias_dqm = detrended_quantile_mapping(observed_data, model_data_prebias, n_quantiles=1000)
corrected_model_data_1st_bias_dqm = detrended_quantile_mapping(observed_data, model_data_1st_bias, n_quantiles=1000)

# Apply distribution mapping
corrected_model_data_prebias_dm = distribution_mapping(observed_data, model_data_prebias, dist=norm)
corrected_model_data_1st_bias_dm = distribution_mapping(observed_data, model_data_1st_bias, dist=norm)

# Apply local intensity scaling
corrected_model_data_prebias_loci = local_intensity_scaling(observed_data, model_data_prebias)
corrected_model_data_1st_bias_loci = local_intensity_scaling(observed_data, model_data_1st_bias)

# Calculate statistics for each method
stats_original_prebias = calculate_statistics(observed_data, model_data_prebias)
stats_eqm_prebias = calculate_statistics(observed_data, corrected_model_data_prebias_eqm)
stats_dqm_prebias = calculate_statistics(observed_data, corrected_model_data_prebias_dqm)
stats_dm_prebias = calculate_statistics(observed_data, corrected_model_data_prebias_dm)
stats_loci_prebias = calculate_statistics(observed_data, corrected_model_data_prebias_loci)

stats_original_1st_bias = calculate_statistics(observed_data, model_data_1st_bias)
stats_eqm_1st_bias = calculate_statistics(observed_data, corrected_model_data_1st_bias_eqm)
stats_dqm_1st_bias = calculate_statistics(observed_data, corrected_model_data_1st_bias_dqm)
stats_dm_1st_bias = calculate_statistics(observed_data, corrected_model_data_1st_bias_dm)
stats_loci_1st_bias = calculate_statistics(observed_data, corrected_model_data_1st_bias_loci)

# Print statistical measures
print(f'Statistics (Original Prebias Model Data): MAE={stats_original_prebias[0]}, MSE={stats_original_prebias[1]}, RMSE={stats_original_prebias[2]}, Correlation={stats_original_prebias[3]}')
print(f'Statistics (Empirical Quantile Mapping Prebias): MAE={stats_eqm_prebias[0]}, MSE={stats_eqm_prebias[1]}, RMSE={stats_eqm_prebias[2]}, Correlation={stats_eqm_prebias[3]}')
print(f'Statistics (Detrended Quantile Mapping Prebias): MAE={stats_dqm_prebias[0]}, MSE={stats_dqm_prebias[1]}, RMSE={stats_dqm_prebias[2]}, Correlation={stats_dqm_prebias[3]}')
print(f'Statistics (Distribution Mapping Prebias): MAE={stats_dm_prebias[0]}, MSE={stats_dm_prebias[1]}, RMSE={stats_dm_prebias[2]}, Correlation={stats_dm_prebias[3]}')
print(f'Statistics (Local Intensity Scaling Prebias): MAE={stats_loci_prebias[0]}, MSE={stats_loci_prebias[1]}, RMSE={stats_loci_prebias[2]}, Correlation={stats_loci_prebias[3]}')

print(f'Statistics (Original 1st Bias Model Data): MAE={stats_original_1st_bias[0]}, MSE={stats_original_1st_bias[1]}, RMSE={stats_original_1st_bias[2]}, Correlation={stats_original_1st_bias[3]}')
print(f'Statistics (Empirical Quantile Mapping 1st Bias): MAE={stats_eqm_1st_bias[0]}, MSE={stats_eqm_1st_bias[1]}, RMSE={stats_eqm_1st_bias[2]}, Correlation={stats_eqm_1st_bias[3]}')
print(f'Statistics (Detrended Quantile Mapping 1st Bias): MAE={stats_dqm_1st_bias[0]}, MSE={stats_dqm_1st_bias[1]}, RMSE={stats_dqm_1st_bias[2]}, Correlation={stats_dqm_1st_bias[3]}')
print(f'Statistics (Distribution Mapping 1st Bias): MAE={stats_dm_1st_bias[0]}, MSE={stats_dm_1st_bias[1]}, RMSE={stats_dm_1st_bias[2]}, Correlation={stats_dm_1st_bias[3]}')
print(f'Statistics (Local Intensity Scaling 1st Bias): MAE={stats_loci_1st_bias[0]}, MSE={stats_loci_1st_bias[1]}, RMSE={stats_loci_1st_bias[2]}, Correlation={stats_loci_1st_bias[3]}')

# Plot the results for visualization
plt.figure(figsize=(24, 20))  # Increase figure size

# Adjust spacing between subplots
plt.subplots_adjust(left=0.1, right=0.9, top=0.95, bottom=0.05, wspace=0.3, hspace=0.4)

# Scatter plot of observed vs model data (Empirical Quantile Mapping Prebias)
plt.subplot(4, 4, 1)
plt.scatter(observed_data, model_data_prebias, alpha=0.5, label='Raw Data Prebias', color='blue')
plt.scatter(observed_data, corrected_model_data_prebias_eqm, alpha=0.5, label='Empirical Quantile Mapping Prebias', color='green')
plt.plot([min(observed_data), max(observed_data)], [min(observed_data), max(observed_data)], 'r--', label='1:1 Line')
plt.xlabel('Observed Temperature (°C)')
plt.ylabel('Model Temperature (°C)')
plt.legend()
plt.title('Empirical Quantile Mapping Prebias')  # Adjusted title

# Line plot of temperatures over time (Empirical Quantile Mapping)
plt.subplot(4, 4, 2)
plt.plot(observed_dates, observed_data, label='Observed Data', marker='.')
plt.plot(model_dates_prebias, corrected_model_data_prebias_eqm, label='Corrected Prebias Data', marker='.')
plt.xlabel('Date')
plt.ylabel('Temperature (°C)')
plt.xticks(rotation=45)  # Rotate x-axis labels
plt.legend()
plt.title('Temperature Over Time (Empirical Quantile Mapping Prebias)')  # Adjusted title

# Scatter plot of observed vs model data (Detrended Quantile Mapping Prebias)
plt.subplot(4, 4, 3)
plt.scatter(observed_data, model_data_prebias, alpha=0.5, label='Raw Data Prebias', color='blue')
plt.scatter(observed_data, corrected_model_data_prebias_dqm, alpha=0.5, label='Detrended Quantile Mapping Prebias', color='orange')
plt.plot([min(observed_data), max(observed_data)], [min(observed_data), max(observed_data)], 'r--', label='1:1 Line')
plt.xlabel('Observed Temperature (°C)')
plt.ylabel('Model Temperature (°C)')
plt.legend()
plt.title('Detrended Quantile Mapping Prebias')  # Adjusted title

# Line plot of temperatures over time (Detrended Quantile Mapping)
plt.subplot(4, 4, 4)
plt.plot(observed_dates, observed_data, label='Observed Data', marker='.')
plt.plot(model_dates_prebias, corrected_model_data_prebias_dqm, label='Corrected Prebias Data', marker='.')
plt.xlabel('Date')
plt.ylabel('Temperature (°C)')
plt.xticks(rotation=45)  # Rotate x-axis labels
plt.legend()
plt.title('Temperature Over Time (Detrended Quantile Mapping Prebias)')  # Adjusted title

# Repeat similar adjustments for other subplots...
# Scatter plot of observed vs model data (Distribution Mapping)
plt.subplot(4, 4, 5)
plt.scatter(observed_data, model_data_prebias, alpha=0.5, label='Raw Data Prebias', color='blue')
plt.scatter(observed_data, corrected_model_data_prebias_dm, alpha=0.5, label='Distribution Mapping Prebias', color='green')
plt.plot([min(observed_data), max(observed_data)], [min(observed_data), max(observed_data)], 'r--', label='1:1 Line')
plt.xlabel('Observed Temperature (°C)')
plt.ylabel('Model Temperature (°C)')
plt.legend()
plt.title('Observed vs Model Temperatures (Distribution Mapping Prebias)')

# Line plot of temperatures over time (Distribution Mapping)
plt.subplot(4, 4, 6)
plt.plot(observed_dates, observed_data, label='Observed Data', marker='.')
plt.plot(model_dates_prebias, corrected_model_data_prebias_dm, label='Corrected Prebias Data', marker='.')
plt.xlabel('Date')
plt.ylabel('Temperature (°C)')
plt.legend()
plt.title('Temperature Over Time (Distribution Mapping Prebias)')

# Scatter plot of observed vs model data (Local Intensity Scaling)
plt.subplot(4, 4, 7)
plt.scatter(observed_data, model_data_prebias, alpha=0.5, label='Raw Data Prebias', color='blue')
plt.scatter(observed_data, corrected_model_data_prebias_loci, alpha=0.5, label='Local Intensity Scaling Prebias', color='green')
plt.plot([min(observed_data), max(observed_data)], [min(observed_data), max(observed_data)], 'r--', label='1:1 Line')
plt.xlabel('Observed Temperature (°C)')
plt.ylabel('Model Temperature (°C)')
plt.legend()
plt.title('Observed vs Model Temperatures (Local Intensity Scaling Prebias)')

# Line plot of temperatures over time (Local Intensity Scaling)
plt.subplot(4, 4, 8)
plt.plot(observed_dates, observed_data, label='Observed Data', marker='o')
plt.plot(model_dates_prebias, corrected_model_data_prebias_loci, label='Corrected Prebias Data', marker='o')
plt.xlabel('Date')
plt.ylabel('Temperature (°C)')
plt.legend()
plt.title('Temperature Over Time (Local Intensity Scaling Prebias)')

plt.subplot(4, 4, 9)
plt.scatter(observed_data, model_data_1st_bias, alpha=0.5, label='Raw Data 1st Bias', color='blue')
plt.scatter(observed_data, corrected_model_data_1st_bias_eqm, alpha=0.5, label='Empirical Quantile Mapping 1st Bias', color='green')
plt.plot([min(observed_data), max(observed_data)], [min(observed_data), max(observed_data)], 'r--', label='1:1 Line')
plt.xlabel('Observed Temperature (°C)')
plt.ylabel('Model Temperature (°C)')
plt.legend()
plt.title('Observed vs Model Temperatures (Empirical Quantile Mapping 1st Bias)')

# Line plot of temperatures over time (Empirical Quantile Mapping)
plt.subplot(4, 4, 10)
plt.plot(observed_dates, observed_data, label='Observed Data', marker='.')
plt.plot(model_dates_1st_bias, corrected_model_data_1st_bias_eqm, label='Corrected 1st Bias Data', marker='.')
plt.xlabel('Date')
plt.ylabel('Temperature (°C)')
plt.legend()
plt.title('Temperature Over Time (Empirical Quantile Mapping 1st Bias)')

# Scatter plot of observed vs model data (Detrended Quantile Mapping)
plt.subplot(4, 4, 11)
plt.scatter(observed_data, model_data_1st_bias, alpha=0.5, label='Raw Data 1st Bias', color='blue')
plt.scatter(observed_data, corrected_model_data_1st_bias_dqm, alpha=0.5, label='Detrended Quantile Mapping 1st Bias', color='green')
plt.plot([min(observed_data), max(observed_data)], [min(observed_data), max(observed_data)], 'r--', label='1:1 Line')
plt.xlabel('Observed Temperature (°C)')
plt.ylabel('Model Temperature (°C)')
plt.legend()
plt.title('Observed vs Model Temperatures (Detrended Quantile Mapping 1st Bias)')

# Line plot of temperatures over time (Detrended Quantile Mapping)
plt.subplot(4, 4, 12)
plt.plot(observed_dates, observed_data, label='Observed Data', marker='.')
plt.plot(model_dates_1st_bias, corrected_model_data_1st_bias_dqm, label='Corrected 1st Bias Data', marker='.')
plt.xlabel('Date')
plt.ylabel('Temperature (°C)')
plt.legend()
plt.title('Temperature Over Time (Detrended Quantile Mapping 1st Bias)')

plt.tight_layout()
plt.show()
